## Building a min GPT with Transformer

> Reference :-
* [Welsh Labs YT Video : How DeepSeek Rewrote the Transformer [MLA]](https://youtu.be/0VLAoVGf_74)
* [Arxiv Paper : DeepSeek-V2: A Strong, Economical, and Efficient Mixture-of-Experts Language Model](https://arxiv.org/pdf/2405.04434)
* [SimpleGPT](SimpleGPT.ipynb)

### Constants

In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(8745)
batch_size = 32 # batch size
block_size = 16 # context length (number of tokens in the input)
embed_size = 32 # embedding size of the Q, K, V
num_heads = 4 # number of heads in the multi-head attention
num_layers = 4 # number of transformer blocks

### Download and Load the Data

In [2]:
# the tiny shakespeare dataset
# check if the file exists
import os
if not os.path.exists("input.txt"):
    print("Downloading the dataset...")
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 

# read the file and display some lines
data = ""
with open('input.txt', 'r') as f:
    data = f.read()

print(f"len of data = {len(data)}\n")
print(data[:100])


len of data = 1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


#### Create the vocabulary from the data (Tokenize)

In [3]:
# create vocabulary
vocab = sorted(list(set(data)))

print("".join(vocab))
vocab_size = len(vocab)
print(f"{vocab_size=}")

# Tokenize
stoi = {char:i for i, char in enumerate(vocab)}
itos = {i:s for s,i in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda s: "".join([itos[c] for c in s]) # you just return a string

print(encode("Hello there"))
print(decode(encode("Hello there")))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65
[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
Hello there


#### Create the Train, Test split and Batching

In [4]:

data_t = torch.tensor(encode(data), dtype=torch.long)
data_t = data_t.to(device)

# train test split
n = int(0.9*len(data_t))
train_data = data_t[:n]
val_data = data_t[n:]

print(f"{len(train_data)=}, {len(val_data)=}")


def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

sample_X, sample_Y = get_batch("train")
print("X", sample_X.shape)
print("Y", sample_Y.shape)

len(train_data)=1003854, len(val_data)=111540
X torch.Size([32, 16])
Y torch.Size([32, 16])


### Model Creation (Multi-Head Latent Attention)

In [14]:
from torch import nn
import torch.nn.functional as F
from torchinfo import summary


@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class MultiHeadLatentAttention(nn.Module):
    def __init__(self, embed_size, num_heads, block_size):
        super().__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size
        self.block_size = block_size
        self.head_size = embed_size // num_heads # 32 / 8 = 4
        self.latent_size = embed_size // 4 # 32 / 8 = 4

        # Down-Project (embed_size -> latent_size) : Shared across heads
        self.W_DQ = nn.Linear(embed_size, self.latent_size, bias=False)
        self.W_DKV = nn.Linear(embed_size, self.latent_size, bias=False)

        # (Concatenated (head_size * num_heads) -> embed_size) : Shared across heads
        self.W_O = nn.Linear(self.head_size * num_heads, embed_size, bias=False)

        # Up-Project (latent_size -> head_size) * num_heads : Head specific
        self.W_UQ = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)
        self.W_UK = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)
        self.W_UV = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)

        # Masking : Shared across heads
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dsqrt = self.head_size ** 0.5

    def forward(self, x):
        B, T, Emb = x.shape

        # Down-Project (embed_size -> latent_size)
        c_Qt = self.W_DQ(x) # B, T, latent_size
        c_KVt = self.W_DKV(x) # B, T, latent_size

        # Up-Project (latent_size -> head_size) * num_heads
        q_Cti = self.W_UQ(c_Qt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size
        k_Cti = self.W_UK(c_KVt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size
        v_Cti = self.W_UV(c_KVt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size

        # (B, T, num_heads, head_size) -> (B, num_heads, T, head_size) for QK matrix multiplication
        q_Cti = q_Cti.permute(0, 2, 1, 3)
        k_Cti = k_Cti.permute(0, 2, 1, 3)
        v_Cti = v_Cti.permute(0, 2, 1, 3)

        # Attention : QK^T / sqrt(head_size)
        att = (q_Cti @ k_Cti.transpose(-2, -1)) / self.dsqrt # B, num_heads, T, T
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        O_ti = att @ v_Cti # B, num_heads, T, head_size

        # B, num_heads, T, head_size -> B, T, num_heads, head_size -> B, T, embed_size
        O_t = O_ti.permute(0, 2, 1, 3).reshape(B, T, -1)
        # (Concatenated (heads * head_size) -> embed_size)
        u_t = self.W_O(O_t)

        return u_t


class FeedForward(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        # hidden size is 4 times the embedding size 
        hidden_size = 4 * embed_size
        # feedforward network
        self.net = nn.Sequential(
            nn.Linear(embed_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, embed_size),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, embed_size, num_heads, block_size):
        super().__init__()
        # multi-head attention layer (series of self-attention heads)
        self.sa = MultiHeadLatentAttention(embed_size, num_heads, block_size)
        self.ff = FeedForward(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class SimpleDeepseek(nn.Module):
    def __init__(self, vocab_size, embed_size, block_size, num_heads, num_layers):
        super().__init__()
        # each token directly maps to a vector
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # each position directly maps to a vector
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # transformer blocks
        self.blocks = nn.Sequential(
            *[Block(embed_size, num_heads, block_size) for _ in range(num_layers)]
        )
        # final layer norm
        self.ln_f = nn.LayerNorm(embed_size)
        # final linear layer to map the embedding to the vocabulary size
        self.lm_head = nn.Linear(embed_size, vocab_size)

    def forward(self, idx, targets=None):
        # B: batch size, T: context length
        B, T = idx.shape
        # token embeddings
        tok_embs = self.token_embedding_table(idx)                          # B, T, Emb
        # position embeddings
        pos_embs = self.position_embedding_table(torch.arange(T, device=idx.device))  # T, Emb
        # add the token and position embeddings
        x = tok_embs + pos_embs                                             # B, T, Emb
        x = self.blocks(x)                                                  # B, T, Emb
        x = self.ln_f(x)                                                    # B, T, Emb
        logits = self.lm_head(x)                                            # B, T, V

        # if targets is None, we are generating
        if targets is None:
            loss = None
        else:
            # reshape the logits and targets to be 2D
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # set the model to evaluation mode
        self.eval()
        for _ in range(max_new_tokens):
            # get the context length (last block_size tokens)
            idx_cond = idx[:, -self.position_embedding_table.num_embeddings:]
            # get the logits and loss
            logits, _ = self(idx_cond)
            # get the last token
            logits = logits[:, -1, :]
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the probabilities
            idx_next = torch.multinomial(probs, num_samples=1)
            # concatenate the new token to the context
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



model = SimpleDeepseek(vocab_size, embed_size, block_size, num_heads, num_layers)
model.to(device)
summary(model, input_data=sample_X , depth=6, row_settings=["var_names"])

Layer (type (var_name))                       Output Shape              Param #
SimpleDeepseek (SimpleDeepseek)               [32, 16, 65]              --
├─Embedding (token_embedding_table)           [32, 16, 32]              2,080
├─Embedding (position_embedding_table)        [16, 32]                  512
├─Sequential (blocks)                         [32, 16, 32]              --
│    └─Block (0)                              [32, 16, 32]              --
│    │    └─LayerNorm (ln1)                   [32, 16, 32]              64
│    │    └─MultiHeadLatentAttention (sa)     [32, 16, 32]              --
│    │    │    └─Linear (W_DQ)                [32, 16, 8]               256
│    │    │    └─Linear (W_DKV)               [32, 16, 8]               256
│    │    │    └─Linear (W_UQ)                [32, 16, 32]              256
│    │    │    └─Linear (W_UK)                [32, 16, 32]              256
│    │    │    └─Linear (W_UV)                [32, 16, 32]              256
│    │    │

#### Model Sample Output without Training

In [11]:
start = torch.zeros((1, 1), dtype=torch.long, device=device)
out = model.generate(start, 1000)
print(decode(out[0].tolist()))


M!:QMpW3UQC'qhQLJLLP;EjdTlcQPDj!Tvy,xDWSpx3dFQUrLVE c'LLYFpPBXiOppLASwL$PJa:pdmvwXDxxeq3EHrL,$!NYWJ
JNdJNqfoV.caqi:poKR-P-TX!FtoJTJo&'wql?TZFerQEPDVSRhHG$m iMcCXPIyQNXm,qeKWk;MhLY
bPZDlbZ!&E'wfNH::Cvqd-GQX ciaoxTToOg.p,g$nXSPHgE::'E
j!:saxcq-xTsVAKAPlkImlylEWkNCHlcWvKWXvedjy,vXScgdI-nCoO-pfLY!tHHfi::KYX:WSlt&$Y:UKfLMHh!YgoQVb&RhOhEBHmPAdbeGs.kNLSjhPhrQY:M.:BLPVwVFKu,dl,ghtJLpyRNDl.HDBgrxw-rLnvB
rjPHhHhfJE3x,HE?i:EUM3!fQ
 H.QEhYdjXw!rWT:EJLgrECyWTlk-Vt dv;NApZlRrv'Qs-rdy.N&dC&w EwrM!HfVutLn:Q,bnzX$tjdDhAhLv.KW&lK.ylSdfJOvPYQT:vPZbwKChqBq$YlVvGmXQKX-kA&DFrLeB
kTt:Z,p,OpPSNLRxp;f!QJKkk-coogXXmAKQD:Ewi,-pJh?:yFJlM WqAH JElf,hEjmLyVyb.fTDKo;SlSlQ-ahjVxLHHC!QoRnC.OWHtHq3HhJor!lyr-md&yHgHbVJE
$JH&A fYDV:NLk squHpSJSv,dZNb.gKO HDqUofYhZf!'JP-ODhmAhx,tiAp!RhuPhnj daSRmXaP!lB,C x'SbqTXcHt&EhHdr:h3
o!pjJn!J.I?PJOsHfYE'TokNUl?!-CmLOhLo??:DQP-LJO;vPwqcQ-xqQ LkXxqDSTSyJOOgLBuTecjYd&YGfwDWb JHt,uqbAQsYplkvDmDfHpEeGQulInlIBHvXJrbAfcthqGFEMvTLxGlCRnr,!GD-KLqxOSKOy.CVHwEWxLcmHvmN Wsqnl!X'wQduApAaDjDNiR

### Train the Model

In [15]:
# Train the model

optimzer = torch.optim.AdamW(model.parameters(), lr=1e-3)

max_iters = 5001
eval_interval = 1000
eval_iters = 100

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(model, eval_iters)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    X, Y = get_batch('train')
    logits, loss = model(X, Y)
    optimzer.zero_grad(set_to_none=True)
    loss.backward()
    optimzer.step()


step 0: train loss 4.2976, val loss 4.2950
step 1000: train loss 2.2632, val loss 2.3015
step 2000: train loss 2.1082, val loss 2.1472
step 3000: train loss 2.0111, val loss 2.0777
step 4000: train loss 1.9562, val loss 2.0370
step 5000: train loss 1.9214, val loss 2.0244


#### Model Sample Output after Training

In [16]:
start = torch.zeros((1, 1), dtype=torch.long, device=device)
out = model.generate(start, 1000)
print(decode(out[0].tolist()))


But wo home of by hims! the his andy the im the will sir; espil'in tcoingers to wor he, sick.

JUMENIUS:
For that me who was surs?

TOUS:
Shixn at op the be succh you a low; and herner at tafy a a houre soullo,

Yephe fich
He suck poten have haves boy. I well jyodly, wore ree, an contlomcul of up? I hi harse
Provet yelland.

MERCHE:
Oo his Fupse is swarn:
Aso your poour; which ball! Had sonmaned heie.

RABENT:
Fareer of my cery the derweer wom that him suing.3 I Ipbo Brovoneror;
This thou of be so house,
Live the sit ince--
Iavin hisoher; with bure, the remanos,
Ans his noot wouldnop your a like,
Ye a night annot: you let
se wor thie
so be off I hap op the word this a not his are leasat I the lot; your that him, ta andramouse, ded rander thogy is our wels: fo gling the ating come
And nightly loover me noblue.

JULORGES:
O where shid leep was thou musat that my hime it his prich,
Arh'lf, ou, hip:
Thou word: our peery, welf undip fling the lrue
Have sh have sills;
That more
Let time bee

---

### Model Creation (MLA with RoPE positional Encoding)

> Reference:-
* [YT : How Rotary Position Embedding Supercharges Modern LLMs RoPE](https://www.youtube.com/watch?v=SMBkImDWOyQ)

In [43]:
from torch import nn
import torch.nn.functional as F
from torchinfo import summary


@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class MultiHeadLatentAttention(nn.Module):
    def __init__(self, embed_size, num_heads, block_size):
        super().__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size
        self.block_size = block_size
        self.head_size = embed_size // num_heads # 32 / 8 = 4
        self.latent_size = embed_size // 8 # 32 / 8 = 4
        self.r_head_size = max(2, self.head_size // 4)

        # Down-Project (embed_size -> latent_size) : Shared across heads
        self.W_DQ = nn.Linear(embed_size, self.latent_size, bias=False)
        self.W_DKV = nn.Linear(embed_size, self.latent_size, bias=False)

        # (Concatenated (head_size * num_heads) -> embed_size) : Shared across heads
        self.W_O = nn.Linear(self.head_size * num_heads, embed_size, bias=False)

        # Up-Project (latent_size -> head_size) * num_heads : Head specific
        self.W_UQ = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)
        self.W_UK = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)
        self.W_UV = nn.Linear(self.latent_size, self.head_size * num_heads, bias=False)

        # Rope Carriers
        self.W_QR = nn.Linear(self.latent_size, self.r_head_size * num_heads, bias=False) # Head specific
        self.W_KR = nn.Linear(self.embed_size, self.r_head_size, bias=False) # Shared across heads

        # Masking : Shared across heads
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.d_sqrt_full = (self.r_head_size + self.head_size) ** 0.5

    def rotary_sin_cos(self, seq_len, base = 10000.0, device=None):
        if device is None:
            device = self.tril.device
        t = torch.arange(seq_len, device=device, dtype=torch.float32)
        inv_freq = 1.0 / (base ** (torch.arange(0, self.r_head_size, 2, device=device, dtype=torch.float32) / self.r_head_size))
        freqs = torch.einsum("t,f->tf", t, inv_freq)
        return freqs.sin(), freqs.cos()

    def apply_rope(self, x, cos, sin):
        # x: B, H, T, D
        x1 = x[..., ::2] # B, H, T, D/2
        x2 = x[..., 1::2] # B, H, T, D/2
        sin = sin.unsqueeze(0).unsqueeze(0)  # 1, 1, T, D/2
        cos = cos.unsqueeze(0).unsqueeze(0)  # 1, 1, T, D/2
        x1_new = (x1 * cos) - (x2 * sin) # B, H, T, D/2
        x2_new = (x1 * sin) + (x2 * cos) # B, H, T, D/2
        return torch.stack((x1_new, x2_new), dim=-1).reshape(x.shape) # B, H, T, D

    def forward(self, x):
        B, T, Emb = x.shape

        # Down-Project (embed_size -> latent_size)
        c_Qt = self.W_DQ(x) # B, T, latent_size
        c_KVt = self.W_DKV(x) # B, T, latent_size

        # Up-Project (latent_size -> head_size) * num_heads
        q_Cti = self.W_UQ(c_Qt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size
        k_Cti = self.W_UK(c_KVt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size
        v_Cti = self.W_UV(c_KVt).view(B, T, self.num_heads, self.head_size) # B, T, num_heads, head_size

        # Rope Carriers
        # Per Head
        q_Rti = self.W_QR(c_Qt).view(B, T, self.num_heads, self.r_head_size) # B, T, num_heads, r_head_size
        # Shared across heads
        k_R = self.W_KR(x) # B, T, r_head_size
        k_Rti = k_R.unsqueeze(2).expand(B, T, self.num_heads, self.r_head_size) # B, T, num_heads, r_head_size

        # (B, T, num_heads, head_size) -> (B, num_heads, T, head_size) for QK matrix multiplication
        q_Cti = q_Cti.permute(0, 2, 1, 3)
        k_Cti = k_Cti.permute(0, 2, 1, 3)
        v_Cti = v_Cti.permute(0, 2, 1, 3)
        q_Rti = q_Rti.permute(0, 2, 1, 3)
        k_Rti = k_Rti.permute(0, 2, 1, 3)

        # Apply Rope
        cos, sin = self.rotary_sin_cos(T)
        q_Rti = self.apply_rope(q_Rti, cos, sin)
        k_Rti = self.apply_rope(k_Rti, cos, sin)

        # Concatenate context and rope carriers
        q_full = torch.cat((q_Cti, q_Rti), dim=-1) # B, T, num_heads, head_size + r_head_size
        k_full = torch.cat((k_Cti, k_Rti), dim=-1) # B, T, num_heads, head_size + r_head_size

        # Attention : QK^T / sqrt(head_size)
        att = (q_full @ k_full.transpose(-2, -1)) / self.d_sqrt_full # B, num_heads, T, T
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        O_ti = att @ v_Cti # B, num_heads, T, head_size

        # B, num_heads, T, head_size -> B, T, num_heads, head_size -> B, T, embed_size
        O_t = O_ti.permute(0, 2, 1, 3).reshape(B, T, -1)
        # (Concatenated (heads * head_size) -> embed_size)
        u_t = self.W_O(O_t)

        return u_t


class FeedForward(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        # hidden size is 4 times the embedding size 
        hidden_size = 4 * embed_size
        # feedforward network
        self.net = nn.Sequential(
            nn.Linear(embed_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, embed_size),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, embed_size, num_heads, block_size):
        super().__init__()
        # multi-head attention layer (series of self-attention heads)
        self.sa = MultiHeadLatentAttention(embed_size, num_heads, block_size)
        self.ff = FeedForward(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class SimpleDeepseek(nn.Module):
    def __init__(self, vocab_size, embed_size, block_size, num_heads, num_layers):
        super().__init__()
        self.block_size = block_size
        # each token directly maps to a vector
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # transformer blocks
        self.blocks = nn.Sequential(
            *[Block(embed_size, num_heads, block_size) for _ in range(num_layers)]
        )
        # final layer norm
        self.ln_f = nn.LayerNorm(embed_size)
        # final linear layer to map the embedding to the vocabulary size
        self.lm_head = nn.Linear(embed_size, vocab_size)

    def forward(self, idx, targets=None):
        # B: batch size, T: context length
        B, T = idx.shape
        # token embeddings
        x = self.token_embedding_table(idx)                          # B, T, Emb
        x = self.blocks(x)                                                  # B, T, Emb
        x = self.ln_f(x)                                                    # B, T, Emb
        logits = self.lm_head(x)                                            # B, T, V

        # if targets is None, we are generating
        if targets is None:
            loss = None
        else:
            # reshape the logits and targets to be 2D
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # set the model to evaluation mode
        self.eval()
        for _ in range(max_new_tokens):
            # get the context length (last block_size tokens)
            idx_cond = idx[:, -self.block_size:]
            # get the logits and loss
            logits, _ = self(idx_cond)
            # get the last token
            logits = logits[:, -1, :]
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the probabilities
            idx_next = torch.multinomial(probs, num_samples=1)
            # concatenate the new token to the context
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



model = SimpleDeepseek(vocab_size, embed_size, block_size, num_heads, num_layers)
model.to(device)
summary(model, input_data=sample_X , depth=6, row_settings=["var_names"])

Layer (type (var_name))                       Output Shape              Param #
SimpleDeepseek (SimpleDeepseek)               [32, 16, 65]              --
├─Embedding (token_embedding_table)           [32, 16, 32]              2,080
├─Sequential (blocks)                         [32, 16, 32]              --
│    └─Block (0)                              [32, 16, 32]              --
│    │    └─LayerNorm (ln1)                   [32, 16, 32]              64
│    │    └─MultiHeadLatentAttention (sa)     [32, 16, 32]              --
│    │    │    └─Linear (W_DQ)                [32, 16, 4]               128
│    │    │    └─Linear (W_DKV)               [32, 16, 4]               128
│    │    │    └─Linear (W_UQ)                [32, 16, 32]              128
│    │    │    └─Linear (W_UK)                [32, 16, 32]              128
│    │    │    └─Linear (W_UV)                [32, 16, 32]              128
│    │    │    └─Linear (W_QR)                [32, 16, 8]               32
│    │    │ 

#### Model Sample Output after Training

In [44]:
# Train the model

optimzer = torch.optim.AdamW(model.parameters(), lr=1e-3)

max_iters = 5001
eval_interval = 1000
eval_iters = 100

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(model, eval_iters)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    X, Y = get_batch('train')
    logits, loss = model(X, Y)
    optimzer.zero_grad(set_to_none=True)
    loss.backward()
    optimzer.step()


step 0: train loss 4.3412, val loss 4.3360
step 1000: train loss 2.2124, val loss 2.2435
step 2000: train loss 2.0881, val loss 2.1416
step 3000: train loss 2.0014, val loss 2.0856
step 4000: train loss 1.9633, val loss 2.0492
step 5000: train loss 1.9185, val loss 2.0353


In [45]:
start = torch.zeros((1, 1), dtype=torch.long, device=device)
out = model.generate(start, 1000)
print(decode(out[0].tolist()))


Ay, sed the awee royay,'dages Wink I wast.
Mas as this greant laing
Hs thee.
QUTABELour IFs, in moulsue depord Ed as an yevexare
Thenderaged of Muusoul ay. fick
Toptih Wipty aloght,
it heat plan a ioun;
Yor, who nondly me thouglang?

LERENCE:
Oncury nowip wet sw ondle, thre.

KING HERY werear,
I'enge no go, thic!

And paser, of take up of minspectencery so, they dime that one shing thall may hEBENCHBRCARLARKE:
Say, piend it a
Dis entest hoir!
Mlakes, home! than and wame thill--dring pupt, and beo,
Of a you,
toou ueg.

LAUTI Call no his hat merie hord good sepowed hy thy rold'd the breart made
Or you;
Of but-sea heir portin thened:
Whose or frect; shiled, proy, Hoods;
That sus your bat lot his will bobow lethinguens
Thy so Of throw he cansty ste clood of ar bread.

Sot lark demph of Beat wid, hous toan, now kif;
Where, so all insitione?

CORIGASCAMIONAOMIY:
And sols shall sug anore.
Bold my wolld, me;
From to hont oftaina-wite foth with shom woult a so.
3
LANG's they that kno him my so